# Controllers and Tasks

This notebook will demonstrate the basic workflow for controllers and tasks.

## Set-Up

First we will generate our trajectories from the benchmark as before.

In [1]:
import autompc as ampc
import numpy as np
from autompc.benchmarks import CartpoleSwingupBenchmark

benchmark = CartpoleSwingupBenchmark()


# Get system and task specification
system = benchmark.system
task   = benchmark.task

# Generate benchmark dataset
trajs = benchmark.gen_trajs(seed=100, n_trajs=10, traj_len=200)

Loading AutoMPC...
Cannot import SnoptWrapper
Cannot import KnitroSolver, make sure its Python interface is installed
Finished loading AutoMPC
running build_ext


Next, we will create a System ID model to be used by the controller.

In [2]:
from autompc.sysid import SINDy

model = SINDy(system, method="lstsq", trig_basis=True, trig_interaction=True)
model.train(trajs)

## Task

Although the benchmark provides a task, we will define a new task here for demonstration.  First, we define the cost function.

In [3]:
from autompc.costs import QuadCost

Q = np.diag([30.0, 3.0, 0.005, 0.1])
R = np.diag([0.02])
F = np.diag([2.0, 3000, 0.15, 0.3])
cost = QuadCost(system, Q, R, F, goal=np.zeros(4))

Now, we initialize the Task with the cost and other properties such as the control bound

In [4]:
from autompc.tasks import Task

task = Task(system)
task.set_cost(cost)
task.set_ctrl_bound("u", -20.0, 20.0)

## Controller

Now, we can create our controller.  Here we demonstrate an iLQR controller with optimization horizon 10

In [5]:
from autompc.control import IterativeLQR

controller = IterativeLQR(system, task, model, horizon=10)

To run the controller, we need to initialize the controller state.  We can do this using the `traj_to_state` method.

In [6]:
# Create stub trajectory
traj = ampc.zeros(system, 1)
traj[0].obs[:] = np.array([3.14, 0.0, 0.0, 0.0])

# Initialize controller state
controller_state = controller.traj_to_state(traj)

We then run the controller by passing the state and the most recent observation.

In [7]:
u, newstate = controller.run(controller_state, np.array([3.14, 0.0, 0.0, 0.0]))
u

array([20.])

By repeatedly calling run, we can simulate the behavior of the controller. To make this easier, we provide the simulate function.

In [8]:
traj = ampc.simulate(controller, init_obs=np.array([3.14, 0.0, 0.0, 0.0]), max_steps=200, dynamics=benchmark.dynamics)

100%|██████████| 200/200 [01:14<00:00,  2.70it/s]


We can the visualize the resulting trajectory

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import HTML

fig = plt.figure()
ax = fig.gca()
anim = benchmark.visualize(fig, ax, traj)
HTML(anim.to_html5_video())